# Data Collection

## All Games
HockeyReference : https://www.hockey-reference.com/ \
All games since 1980 avaible as csv - No actual scraping required

### Cleaning Games Dataset

In [ ]:
# Import the pandas library as pd
import pandas as pd

# Read the csv file "Regular Season Games.txt" into a pandas DataFrame called "df"
df = pd.read_csv("Regular Season Games.txt")

# Drop the 5th, 6th, 7th, and 8th columns of the DataFrame
df = df.drop(df.columns[[5, 6, 7, 8]], axis=1)

# Create a new column called "Winner" that contains the name of the winning team
# If the "G" column is greater than the "G.1" column, the winner is the "Visitor"
# If the "G" column is less than the "G.1" column, the winner is the "Home"
# If the "G" column is equal to the "G.1" column, the winner is considered a tie
df['Winner'] = df.apply(lambda x: x['Visitor'] if x['G'] > x['G.1'] else (x['Home'] if x['G'] < x['G.1'] else 'Tie'), axis=1)

# Filter out any rows where the winner is a tie
df = df[df['Winner'] != 'Tie']


In [ ]:
# Import the datetime module from the Python standard library
from datetime import datetime

# Define a function called "closest_year" that takes a string in the format "YYYY-MM-DD" as input
def closest_year(date_str):
    # Convert the string into a datetime object using the strptime method
    date = datetime.strptime(date_str, "%Y-%m-%d")
    
    # Check if the month of the datetime object is less than 7 (July)
    if date.month < 7:
        # If it is, return the year of the datetime object
        return date.year
    else:
        # If it isn't, return the year of the datetime object plus one
        return date.year + 1


In [ ]:
# Create a new column "Date" in the DataFrame "df" by applying the function "closest_year" to each row along the axis 1 (columns)
df['Season'] = df.apply(lambda x: closest_year(x['Date']), axis = 1)

In [ ]:
df = df[df['Season'] != 1995]

### Standings
NHL.Com : https://www.nhl.com/ \
Standings from seasons from 1993 - 2022 : Available as csv

### Reading in Home and Away Stats for Each time in Every Game

#### Creating Consistent Column Structure

In [ ]:
# Read the csv file "Standings.csv" into a pandas DataFrame called "df1"
df1 = pd.read_csv("Standings.csv")

# Create a new column "Season" by taking the modulo of each value in the "Season" column with 10000
df1['Season'] =  df1['Season'] % 10000


df1 = df1.drop(columns=["FOW%", "OT", "T"])



#### Dealing With Naming Convention Differences

In [ ]:
#Helper
def replace_value(x):
    if isinstance(x, str) and x == "Montréal Canadiens":
        x = x.replace("Montréal Canadiens", "Montreal Canadiens")
        return x
    else:
        return x

df1 = df1.applymap(replace_value)


In [ ]:
# Helper
def replace_value(x):
    if isinstance(x, str) and x == "Winnipeg Jets (1979)":
        x = x.replace("Winnipeg Jets (1979)", "Winnipeg Jets")
        return x
    else:
        return x

df1 = df1.applymap(replace_value)


In [ ]:
import numpy as np
mask = df1['Season'] <= 2004

df1['Team'] = np.where(mask, df1['Team'].str.replace("Anaheim Ducks", "Mighty Ducks of Anaheim"), df1['Team'])

#### Creating Merged Dataframe with Home and Visitor Stats

In [ ]:
# merge the dataframes
merged_df1 = pd.merge(df, df1, left_on=['Season', 'Visitor'], right_on=['Season', 'Team'])

# rename the columns from the other dataframe
merged_df1 = merged_df1.rename(columns={'GP': 'v_GP', 'W': 'v_W', 'L': 'v_L', 'P': 'v_P', 'P%': 'v_P%', 'RW': 'v_RW', 'ROW': 'v_ROW', 'S/O Win': 'v_S/O Win', 'GF': 'v_GF', 'GA': 'v_GA', 'GF/GP': 'v_GF/GP', 'GA/GP': 'v_GA/GP', 'PP%': 'v_PP%', 'PK%': 'v_PK%', 'Net PP%': 'v_Net PP%', 'Net PK%': 'v_Net PK%', 'Shots/GP': 'v_Shots/GP', 'SA/GP': 'v_SA/GP'})


# merge the dataframes again
merged_df2 = pd.merge(merged_df1, df1, left_on=['Season', 'Home'], right_on=['Season', 'Team'])

# rename the columns from the other dataframe
merged_df2 = merged_df2.rename(columns={'GP': 'h_GP', 'W': 'h_W', 'L': 'h_L', 'P': 'h_P', 'P%': 'h_P%', 'RW': 'h_RW', 'ROW': 'h_ROW', 'S/O Win': 'h_S/O Win', 'GF': 'h_GF', 'GA': 'h_GA', 'GF/GP': 'h_GF/GP', 'GA/GP': 'h_GA/GP', 'PP%': 'h_PP%', 'PK%': 'h_PK%', 'Net PP%': 'h_Net PP%', 'Net PK%': 'h_Net PK%', 'Shots/GP': 'h_Shots/GP', 'SA/GP': 'h_SA/GP'})



#### Final Cleaning Touches

In [ ]:
final_df = merged_df2.drop(columns = ['Visitor', 'Home'])

In [ ]:
final_df = final_df.rename(columns = {'G' : 'v_goals', 'G.1' : 'h_goals', 'Team_x' : 'Visitor', 'Team_y' : 'Home', 'Season_x' : 'Season'})

In [ ]:
final_df = final_df.sort_values('Date')

In [ ]:
final_df.to_csv("ArtifactDataset")